### Libraries

In [84]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

In [79]:
df = pd.read_csv("C:\\Users\\AJIT\\Desktop\\santander-customer-transaction-prediction\\train.csv")
df_test = pd.read_csv("C:\\Users\\AJIT\\Desktop\\santander-customer-transaction-prediction\\test.csv")

In [3]:
df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
df.shape

(200000, 202)

In [80]:
df['Type']='Train'
df_test['Type']='Test'
data = pd.concat([df,df_test],axis=0)

C:\Users\AJIT\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [81]:
ID_col = ['ID_code']
target_col = ["target"]
other_col=['Type'] #Test and Train Data set identifier
num_cols= list(set(list(data.columns))-set(ID_col)-set(target_col)-set(other_col))

In [66]:
#Target variable is also a categorical so convert it
data["target"] =data["target"].astype('str')

train=data[data['Type']=='Train']
test=data[data['Type']=='Test']

train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]
 

C:\Users\AJIT\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [82]:
features=list(set(list(data.columns))-set(ID_col)-set(target_col)-set(other_col))

In [68]:
x_train = Train[list(features)].values
y_train = Train["target"].values
x_validate = Validate[list(features)].values
y_validate = Validate["target"].values
x_test=test[list(features)].values

In [8]:
y_validate

array(['0.0', '1.0', '0.0', ..., '0.0', '0.0', '0.0'], dtype=object)

### PCA

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_validate = scaler.transform(x_validate)

In [10]:
x_test= scaler.transform(x_test)

In [11]:
from sklearn.decomposition import PCA
pca = PCA(.80)

In [12]:
pca.fit(x_train)
x_train = pca.transform(x_train)
x_validate= pca.transform(x_validate)

In [13]:
x_test= pca.transform(x_test)

In [14]:
x_train.shape

(149941, 158)

### RFECV

In [71]:
#https://chrisalbon.com/machine_learning/feature_selection/recursive_feature_elimination/
#https://www.scikit-yb.org/en/latest/api/features/rfecv.html
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

In [72]:
model = LogisticRegression()
rfecv = RFECV(estimator=model,scoring="roc_auc")

In [85]:
#rfecv.fit(x_train, y_train)
#rfecv.transform(x_train)

In [ ]:
rfecv.support_

In [ ]:
cols = [2,9,16,26,69,72,81,84,93,94,101]
X_train.drop(X_train.columns[cols],axis=1,inplace=True)
X_test.drop(X_test.columns[cols],axis=1,inplace=True)

### Bagging meta-estimator

In [85]:
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
bme= BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
bme.fit(x_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=None, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [86]:
probs = bme.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.803


### Random Forest

In [69]:
random.seed(100)
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [70]:
probs = rf.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.701


### GBM

In [60]:
gbm= GradientBoostingClassifier()
gbm.fit(x_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [76]:
probs = gbm.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.859


In [66]:
final_status = gbm.predict_proba(x_test)
df_test["target"]=final_status[:,1]
df_test.to_csv("gbm.csv",columns=['ID_code','target'])

### Ada Boost

In [67]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(x_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [74]:
probs = ada.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.856


In [78]:
final_status = ada.predict_proba(x_test)
df_test["target"]=final_status[:,1]
df_test.to_csv("ada.csv",columns=['ID_code','target'])

### Light GBM

In [76]:
import lightgbm as lgb
lg = lgb.LGBMClassifier(silent=False)
lg.fit(x_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [77]:
probs = lg.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.865


In [83]:
final_status = lg.predict_proba(x_test)
df_test["target"]=final_status[:,1]
df_test.to_csv("lg.csv",columns=['ID_code','target'])

### Xg Boost

In [75]:
import xgboost as xgb
xg=xgb.XGBClassifier()
xg.fit(x_train, y_train)

KeyboardInterrupt: 

In [89]:
probs = xg.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.858


### Cat Boost

In [57]:
import catboost as cb
cbo=cb.CatBoostClassifier()
clf = cb.CatBoostClassifier()
clf.fit(x_train,y_train)

Learning rate set to 0.06914
0:	learn: 0.6336112	total: 2.5s	remaining: 41m 43s
1:	learn: 0.5837449	total: 3.43s	remaining: 28m 30s
2:	learn: 0.5420037	total: 3.85s	remaining: 21m 19s
3:	learn: 0.5071254	total: 4.25s	remaining: 17m 39s
4:	learn: 0.4773178	total: 4.53s	remaining: 15m 1s
5:	learn: 0.4535069	total: 4.81s	remaining: 13m 17s
6:	learn: 0.4323263	total: 5.23s	remaining: 12m 22s
7:	learn: 0.4142300	total: 5.53s	remaining: 11m 26s
8:	learn: 0.3993409	total: 5.86s	remaining: 10m 44s
9:	learn: 0.3859029	total: 6.38s	remaining: 10m 32s
10:	learn: 0.3751170	total: 6.69s	remaining: 10m 1s
11:	learn: 0.3649890	total: 6.94s	remaining: 9m 31s
12:	learn: 0.3569453	total: 7.33s	remaining: 9m 16s
13:	learn: 0.3493594	total: 7.64s	remaining: 8m 57s
14:	learn: 0.3426829	total: 8.21s	remaining: 8m 58s
15:	learn: 0.3373662	total: 8.49s	remaining: 8m 42s
16:	learn: 0.3324897	total: 8.8s	remaining: 8m 28s
17:	learn: 0.3284743	total: 9.07s	remaining: 8m 14s
18:	learn: 0.3246656	total: 9.36s	rema

157:	learn: 0.2447868	total: 47.7s	remaining: 4m 13s
158:	learn: 0.2445573	total: 47.9s	remaining: 4m 13s
159:	learn: 0.2443093	total: 48.2s	remaining: 4m 12s
160:	learn: 0.2440664	total: 48.5s	remaining: 4m 12s
161:	learn: 0.2438173	total: 48.7s	remaining: 4m 11s
162:	learn: 0.2435752	total: 48.9s	remaining: 4m 11s
163:	learn: 0.2433621	total: 49.1s	remaining: 4m 10s
164:	learn: 0.2431091	total: 49.4s	remaining: 4m 10s
165:	learn: 0.2429114	total: 49.6s	remaining: 4m 9s
166:	learn: 0.2426838	total: 49.9s	remaining: 4m 8s
167:	learn: 0.2424438	total: 50.1s	remaining: 4m 8s
168:	learn: 0.2421929	total: 50.3s	remaining: 4m 7s
169:	learn: 0.2419863	total: 50.5s	remaining: 4m 6s
170:	learn: 0.2417563	total: 50.8s	remaining: 4m 6s
171:	learn: 0.2415410	total: 51s	remaining: 4m 5s
172:	learn: 0.2413080	total: 51.2s	remaining: 4m 4s
173:	learn: 0.2410859	total: 51.5s	remaining: 4m 4s
174:	learn: 0.2408780	total: 51.7s	remaining: 4m 3s
175:	learn: 0.2406731	total: 51.9s	remaining: 4m 2s
176:	l

312:	learn: 0.2184576	total: 1m 30s	remaining: 3m 17s
313:	learn: 0.2183331	total: 1m 30s	remaining: 3m 17s
314:	learn: 0.2182138	total: 1m 30s	remaining: 3m 17s
315:	learn: 0.2180778	total: 1m 31s	remaining: 3m 17s
316:	learn: 0.2179653	total: 1m 31s	remaining: 3m 17s
317:	learn: 0.2178569	total: 1m 31s	remaining: 3m 16s
318:	learn: 0.2177542	total: 1m 32s	remaining: 3m 16s
319:	learn: 0.2176330	total: 1m 32s	remaining: 3m 16s
320:	learn: 0.2175269	total: 1m 32s	remaining: 3m 16s
321:	learn: 0.2173866	total: 1m 33s	remaining: 3m 15s
322:	learn: 0.2172650	total: 1m 33s	remaining: 3m 15s
323:	learn: 0.2171460	total: 1m 33s	remaining: 3m 15s
324:	learn: 0.2170395	total: 1m 33s	remaining: 3m 15s
325:	learn: 0.2169148	total: 1m 34s	remaining: 3m 14s
326:	learn: 0.2167999	total: 1m 34s	remaining: 3m 14s
327:	learn: 0.2166928	total: 1m 34s	remaining: 3m 14s
328:	learn: 0.2165816	total: 1m 35s	remaining: 3m 14s
329:	learn: 0.2164562	total: 1m 35s	remaining: 3m 14s
330:	learn: 0.2163353	total:

466:	learn: 0.2033728	total: 2m 12s	remaining: 2m 30s
467:	learn: 0.2032972	total: 2m 12s	remaining: 2m 30s
468:	learn: 0.2032218	total: 2m 12s	remaining: 2m 30s
469:	learn: 0.2031379	total: 2m 12s	remaining: 2m 29s
470:	learn: 0.2030612	total: 2m 13s	remaining: 2m 29s
471:	learn: 0.2029901	total: 2m 13s	remaining: 2m 29s
472:	learn: 0.2029135	total: 2m 13s	remaining: 2m 28s
473:	learn: 0.2028372	total: 2m 13s	remaining: 2m 28s
474:	learn: 0.2027673	total: 2m 13s	remaining: 2m 28s
475:	learn: 0.2026848	total: 2m 14s	remaining: 2m 27s
476:	learn: 0.2026150	total: 2m 14s	remaining: 2m 27s
477:	learn: 0.2025417	total: 2m 14s	remaining: 2m 27s
478:	learn: 0.2024645	total: 2m 14s	remaining: 2m 26s
479:	learn: 0.2023870	total: 2m 15s	remaining: 2m 26s
480:	learn: 0.2022996	total: 2m 15s	remaining: 2m 26s
481:	learn: 0.2022545	total: 2m 15s	remaining: 2m 25s
482:	learn: 0.2021738	total: 2m 15s	remaining: 2m 25s
483:	learn: 0.2020919	total: 2m 16s	remaining: 2m 25s
484:	learn: 0.2020226	total:

619:	learn: 0.1928781	total: 2m 50s	remaining: 1m 44s
620:	learn: 0.1928377	total: 2m 51s	remaining: 1m 44s
621:	learn: 0.1927879	total: 2m 51s	remaining: 1m 44s
622:	learn: 0.1927286	total: 2m 51s	remaining: 1m 43s
623:	learn: 0.1926770	total: 2m 51s	remaining: 1m 43s
624:	learn: 0.1926174	total: 2m 52s	remaining: 1m 43s
625:	learn: 0.1925505	total: 2m 52s	remaining: 1m 42s
626:	learn: 0.1925002	total: 2m 52s	remaining: 1m 42s
627:	learn: 0.1924432	total: 2m 52s	remaining: 1m 42s
628:	learn: 0.1923646	total: 2m 53s	remaining: 1m 42s
629:	learn: 0.1923108	total: 2m 53s	remaining: 1m 41s
630:	learn: 0.1922489	total: 2m 53s	remaining: 1m 41s
631:	learn: 0.1921903	total: 2m 53s	remaining: 1m 41s
632:	learn: 0.1921360	total: 2m 54s	remaining: 1m 40s
633:	learn: 0.1920782	total: 2m 54s	remaining: 1m 40s
634:	learn: 0.1920314	total: 2m 54s	remaining: 1m 40s
635:	learn: 0.1919724	total: 2m 54s	remaining: 1m 40s
636:	learn: 0.1919244	total: 2m 55s	remaining: 1m 39s
637:	learn: 0.1918622	total:

773:	learn: 0.1844272	total: 3m 30s	remaining: 1m 1s
774:	learn: 0.1843822	total: 3m 30s	remaining: 1m 1s
775:	learn: 0.1843301	total: 3m 30s	remaining: 1m
776:	learn: 0.1842918	total: 3m 30s	remaining: 1m
777:	learn: 0.1842484	total: 3m 30s	remaining: 1m
778:	learn: 0.1842079	total: 3m 31s	remaining: 59.9s
779:	learn: 0.1841626	total: 3m 31s	remaining: 59.6s
780:	learn: 0.1841110	total: 3m 31s	remaining: 59.4s
781:	learn: 0.1840592	total: 3m 31s	remaining: 59.1s
782:	learn: 0.1840151	total: 3m 32s	remaining: 58.8s
783:	learn: 0.1839792	total: 3m 32s	remaining: 58.5s
784:	learn: 0.1839332	total: 3m 32s	remaining: 58.2s
785:	learn: 0.1838827	total: 3m 32s	remaining: 58s
786:	learn: 0.1838320	total: 3m 33s	remaining: 57.7s
787:	learn: 0.1837807	total: 3m 33s	remaining: 57.4s
788:	learn: 0.1837409	total: 3m 33s	remaining: 57.1s
789:	learn: 0.1836978	total: 3m 34s	remaining: 56.9s
790:	learn: 0.1836485	total: 3m 34s	remaining: 56.6s
791:	learn: 0.1836085	total: 3m 34s	remaining: 56.4s
792:

930:	learn: 0.1776113	total: 4m 18s	remaining: 19.1s
931:	learn: 0.1775636	total: 4m 18s	remaining: 18.9s
932:	learn: 0.1775185	total: 4m 18s	remaining: 18.6s
933:	learn: 0.1774794	total: 4m 19s	remaining: 18.3s
934:	learn: 0.1774422	total: 4m 19s	remaining: 18s
935:	learn: 0.1773928	total: 4m 20s	remaining: 17.8s
936:	learn: 0.1773562	total: 4m 20s	remaining: 17.5s
937:	learn: 0.1773269	total: 4m 20s	remaining: 17.2s
938:	learn: 0.1772871	total: 4m 21s	remaining: 17s
939:	learn: 0.1772375	total: 4m 21s	remaining: 16.7s
940:	learn: 0.1771994	total: 4m 21s	remaining: 16.4s
941:	learn: 0.1771483	total: 4m 22s	remaining: 16.1s
942:	learn: 0.1771122	total: 4m 22s	remaining: 15.9s
943:	learn: 0.1770813	total: 4m 22s	remaining: 15.6s
944:	learn: 0.1770316	total: 4m 22s	remaining: 15.3s
945:	learn: 0.1769892	total: 4m 23s	remaining: 15s
946:	learn: 0.1769571	total: 4m 23s	remaining: 14.8s
947:	learn: 0.1769261	total: 4m 24s	remaining: 14.5s
948:	learn: 0.1768847	total: 4m 24s	remaining: 14.2s

In [58]:
probs = clf.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.891


### Tuning

In [ ]:
params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300]}
cb = cb.CatBoostClassifier()
cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
cb_model.fit(x_train, y_train)

In [59]:
final_status = clf.predict_proba(x_test)
df_test["target"]=final_status[:,1]
df_test.to_csv("clf.csv",columns=['ID_code','target'])

### H2O

In [51]:
x_train=pd.DataFrame(x_train)
y_train=pd.DataFrame(y_train)
x_validate=pd.DataFrame(x_validate)
y_validate=pd.DataFrame(y_validate)
x_test=pd.DataFrame(x_test)

In [52]:
df_train=pd.concat([x_train,y_train],axis=1)
df_val=pd.concat([x_validate,y_validate],axis=1)


In [53]:
df_train.to_csv("df_train.csv")
df_val.to_csv("df_val.csv")

In [54]:
import h2o
import numpy as np
import math
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init(nthreads=-1, strict_version_check=True)
## optional: connect to a running H2O cluster
#h2o.init(ip="mycluster", port=55555)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,34 mins 45 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.4
H2O cluster version age:,1 month and 17 days
H2O cluster name:,H2O_from_python_AJIT_gxc13r
H2O cluster total nodes:,1
H2O cluster free memory:,532 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [55]:
## 'path' can point to a local file, hdfs, s3, nfs, Hive, directories, etc.
df = h2o.import_file(path = "C:\\Users\\AJIT\\Desktop\\df_train.csv")
df_val = h2o.import_file(path ="C:\\Users\\AJIT\\Desktop\\df_val.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [56]:
df.head()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C101,C102,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112,C113,C114,C115,C116,C117,C118,C119,C120,C121,C122,C123,C124,C125,C126,C127,C128,C129,C130,C131,C132,C133,C134,C135,C136,C137,C138,C139,C140,C141,C142,C143,C144,C145,C146,C147,C148,C149,C150,C151,C152,C153,C154,C155,C156,C157,C158,C159,C160,C161,C162,C163,C164,C165,C166,C167,C168,C169,C170,C171,C172,C173,C174,C175,C176,C177,C178,C179,C180,C181,C182,C183,C184,C185,C186,C187,C188,C189,C190,C191,C192,C193,C194,C195,C196,C197,C198,C199,C200
nan,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198
0,31.4045,8.6989,6.7806,18.3816,-19.7159,12.9041,5.8056,17.9244,-3.5189,12.1749,3.8743,-0.2154,0.5857,9.3225,3.9642,15.4462,16.1828,-4.5213,1.691,9.6508,11.2668,9.7268,17.1554,11.9081,-1.9245,-1.0914,7.8784,-13.4221,24.3627,1.3675,-3.4132,13.6468,3.1364,0.8897,14.0137,9.9913,-5.1488,8.3694,14.71,12.0039,5.7487,8.7989,29.046,-0.694,8.9255,3.6159,13.4325,12.9194,5.1187,8.86,2.8279,4.6667,4.9306,14.3831,2.524,-6.7863,11.2663,9.5981,8.3307,0.6764,12.0446,16.2572,18.5177,16.0094,3.1345,-4.7645,-4.92,-0.3085,21.6374,0.5663,-1.7069,12.1983,0.1812,1.4403,10.5785,-4.701,10.535,6.7602,16.0283,4.0288,16.8911,2.7004,11.5423,5.3822,1.7202,15.6599,9.5319,-13.1324,7.7423,11.0924,2.9423,6.676,5.3133,22.4321,-0.7474,4.284,3.0657,-9.2834,5.4879,43.1127,0.5745,11.5767,29.4547,2.0771,18.6266,3.9141,-0.6975,25.8398,-8.4051,9.0056,11.5659,6.5674,11.8411,4.959,5.1726,9.4763,-0.4073,30.7133,-5.2387,-0.4851,13.7241,11.192,14.8421,18.5227,6.5199,16.4597,19.0614,14.5691,-16.4727,5.0384,13.8571,8.9642,-3.8766,15.4576,2.1743,17.5743,2.9252,10.78,4.4354,-14.2136,4.2218,0.3587,2.5516,2.4716,-10.1245,13.8372,23.4104,14.1104,10.4838,5.9226,-21.4494,12.6644,26.5376,14.8322,5.1999,2.0323,1.5093,12.8287,17.6054,7.8871,21.4669,5.1736,2.0183,-0.7338,5.0109,-8.4254,-2.3978,8.5635,1.6573,5.3253,5.747,18.5618,6.0454,0.7754,5.8764,-5.6011,8.6748,-7.2393,-7.017,-2.344,7.9984,20.8773,16.6964,3.1531,14.3691,9.0164,12.7803,16.2191,5.093,10.1166,5.9965,2.5791,9.5426,13.3102,15.5752,11.4607,7.3746,11.6418,3.1821
1,18.1577,8.6086,6.8852,7.0529,-15.9319,7.8755,5.4007,-4.4223,-7.3477,12.2419,3.6783,-1.3147,-0.3566,-13.5901,7.7214,16.5346,8.0109,-5.485,10.9516,5.7582,19.3219,9.5018,32.0888,13.8588,0.8194,1.9518,8.1267,-11.51,13.0858,0.3683,0.6939,2.5462,2.5837,8.2736,14.0239,10.9631,-11.7684,11.0738,15.1866,12.5942,13.8275,5.4345,13.2256,-4.1083,11.5006,3.8208,13.891,11.8343,5.6208,20.1068,-1.9721,10.816,5.0464,6.9779,3.6159,-4.1473,12.2806,3.8821,3.6937,2.7995,11.7842,26.6881,20.6294,31.5899,-8.9921,5.5378,3.1468,-9.2987,40.5632,0.5134,12.2301,13.7814,-1.2722,1.7093,0.7399,2.2302,3.4287,7.1051,34.4014,4.2578,19.371,3.2003,10.6261,7.0118,11.0752,16.1622,1.5365,-9.6953,10.4968,8.0905,-4.821,-5.0121,5.8683,7.9344,-8.0946,7.8,-14.91,7.0433,5.7999,7.7841,8.4135,9.8775,9.7797,1.7944,16.5338,5.3523,9.3192,22.5441,-5.5937,9.4317,8.9231,7.0513,7.6543,-7.2417,1.9568,-13.695,-2.5586,28.5708,-11.1147,8.5426,16.5721,11.3702,11.7569,15.430

In [37]:
response = "C160"
df[response] = df[response].asfactor()           
df_val[response] = df_val[response].asfactor()           
predictors = df.columns
predictors[1:156]

['C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26',
 'C27',
 'C28',
 'C29',
 'C30',
 'C31',
 'C32',
 'C33',
 'C34',
 'C35',
 'C36',
 'C37',
 'C38',
 'C39',
 'C40',
 'C41',
 'C42',
 'C43',
 'C44',
 'C45',
 'C46',
 'C47',
 'C48',
 'C49',
 'C50',
 'C51',
 'C52',
 'C53',
 'C54',
 'C55',
 'C56',
 'C57',
 'C58',
 'C59',
 'C60',
 'C61',
 'C62',
 'C63',
 'C64',
 'C65',
 'C66',
 'C67',
 'C68',
 'C69',
 'C70',
 'C71',
 'C72',
 'C73',
 'C74',
 'C75',
 'C76',
 'C77',
 'C78',
 'C79',
 'C80',
 'C81',
 'C82',
 'C83',
 'C84',
 'C85',
 'C86',
 'C87',
 'C88',
 'C89',
 'C90',
 'C91',
 'C92',
 'C93',
 'C94',
 'C95',
 'C96',
 'C97',
 'C98',
 'C99',
 'C100',
 'C101',
 'C102',
 'C103',
 'C104',
 'C105',
 'C106',
 'C107',
 'C108',
 'C109',
 'C110',
 'C111',
 'C112',
 'C113',
 'C114',
 'C115',
 'C116',
 'C117',
 'C118',
 'C119',
 'C120',
 'C121',
 'C122',
 'C123',
 'C124',

In [38]:
gbm = H2OGradientBoostingEstimator()
gbm.train(x=predictors, y=response, training_frame=df)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [40]:
perf = gbm.model_performance(df_val)
perf.auc()

0.8562849093618705

### Random Forest

In [41]:
#We only provide the required parameters, everything else is default
rf = h2o.estimators.random_forest.H2ORandomForestEstimator()
rf.train(x=predictors, y=response, training_frame=df)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [42]:
perf = rf.model_performance(df_val)
perf.auc()

0.837884031467246